In [1]:
import pandas as pd
import numpy as np
import time
import resource
from os import walk
from datetime  import datetime,timedelta,date
import multiprocessing as mp
import os
import re
import subprocess
import matplotlib.pyplot as plt
# from tld import get_tld
import shutil
import csv
import seaborn as sns
import pickle
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

def save_obj(obj, file_name):    
    file_path = 'pyobj/' + file_name
    file_save = open(file_path, 'wb')
    pickle.dump(obj, file_save, pickle.HIGHEST_PROTOCOL)
    file_save.close()

def load_obj(file_name):
    path = 'pyobj/' + file_name
    file_save = open(path, 'rb')
    return pickle.load(file_save)

In [2]:
def read_data_file(f):
    s = open(f, 'r').read().split('|')
    song_type = s[0]
    song_data = s[1]
    
    return (song_type, song_data)

In [9]:
types = []

f = os.walk('/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song')
for fi in f:
    f = fi
    break
for fi in f[2]:
    types.append(read_data_file(f[0]+'/'+fi)[0])

In [10]:
df = pd.DataFrame(pd.Series(types)).rename(columns={0:'type'})
df['freq'] = 1
df.groupby('type').agg({'freq':'sum'}).reset_index().sort_values('freq', ascending=False)

,type,freq
18,Trữ Tình,7398
8,Nhạc Trẻ,2845
4,Không Lời,1389
13,Remix Việt,881
10,Pop,810
2,Cách Mạng,670
11,R&B/Hip Hop/Rap,503
16,Thể Loại Khác,499
3,Electronica/Dance,412
17,Tiền Chiến,390
